In [28]:
!pip install transformers
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import pandas as pd
import re

df = pd.read_excel('/content/drive/My Drive/HASOC/hasoc_2020_en_train_new_A.xlsx')

df_test = pd.read_csv('/content/drive/My Drive/HASOC/english_test_1509.csv')

output_label = 'task2'

#STOPWORDS = set(stopwords.words('english'))
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
LINKS =re.compile('(?i)(http(s)?[:/.a-z0-9]*)')
CHAR_OCC = re.compile(r'(.)\1{2,}')
LINKS = re.compile("(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")
def preprocess_tweet(text):
    text = text.lower() # lowercase text
    text = LINKS.sub('', text)
    text = CHAR_OCC.sub(r'\1',text) # replace more than 2 occurences of a same char like aaaaabbbbbbb to ab
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = re.sub("@[a-zA-Z]*", "", text) # remove usernames with @******
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub("\'", "", text) # remove backslash-apostrophe 
    text = re.sub("[^a-zA-Z]"," ",text) # remove everything except alphabets
    text = re.sub("( |^)[a-zA-Z][a-zA-Z]? "," ",text) # remove words with just 1 or 2 letters
    text = ' '.join(text.split()) # remove whitespaces 
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

df['text'] = df['text'].apply(lambda x: preprocess_tweet(x))   
df_test['text'] = df_test['text'].apply(lambda x: preprocess_tweet(x))  
df[output_label].value_counts()


NONE    1852
PRFN    1377
OFFN     321
HATE     158
Name: task2, dtype: int64

In [29]:
possible_labels = df[output_label].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

df['label'] = df[output_label].replace(label_dict)
df_test['label'] = df_test[output_label].replace(label_dict)
df.head()

,tweet_id,text,task1,task2,ID,label
0,1123757263427186690,hate wen females hit nigga with tht bro tryna ...,HOF,PRFN,hasoc_2020_en_2574,0
1,1123733301397733380,when youre from the bay but youre really ny ni...,HOF,PRFN,hasoc_2020_en_3627,0
2,1123734094108659712,dear democrats the american people arent stupi...,NOT,NONE,hasoc_2020_en_3108,1
3,1126951188170199049,aint drugs just bored be doing the same shit w...,HOF,PRFN,hasoc_2020_en_3986,0
4,1126863510447710208,summer coming for you boring shit beach days r...,NOT,NONE,hasoc_2020_en_5152,1


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby([output_label, 'label', 'data_type']).count()

tweet_id  text  task1    ID
task2 label data_type                             
HATE  3     train           134   134    134   134
            val              24    24     24    24
NONE  1     train          1574  1574   1574  1574
            val             278   278    278   278
OFFN  2     train           273   273    273   273
            val              48    48     48    48
PRFN  0     train          1170  1170   1170  1170
            val             207   207    207   207

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df_test.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitely activated but `max_length` is provided a specific value, please use

In [32]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False).to('cuda')

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [34]:
device='cuda'
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to('cuda') for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/My Drive/HASOC/english/task2/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.8439960544029487
Validation loss: 0.8097645033311139
F1 Score (Weighted): 0.7706954777934353



Epoch 2
Training loss: 0.6918869351420541
Validation loss: 0.7185011764635302
F1 Score (Weighted): 0.8017374735836829



Epoch 3
Training loss: 0.5587643682365204
Validation loss: 0.7548922463958603
F1 Score (Weighted): 0.8028388205785973



Epoch 4
Training loss: 0.4092520091623523
Validation loss: 0.8998113716101294
F1 Score (Weighted): 0.8020108013244082



Epoch 5
Training loss: 0.3450846407979197
Validation loss: 0.9883223358117375
F1 Score (Weighted): 0.7942278388956315



In [35]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/drive/My Drive/HASOC/english/task2/finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: PRFN
Accuracy: 180/207

Class: NONE
Accuracy: 243/278

Class: OFFN
Accuracy: 17/48

Class: HATE
Accuracy: 4/24



In [36]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

_, predictions_test, true_test = evaluate(dataloader_test)


def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(classification_report(labels_flat, preds_flat))

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

accuracy_per_class(predictions_test, true_test)

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       293
           1       0.91      0.92      0.91       414
           2       0.44      0.33      0.38        82
           3       0.22      0.16      0.19        25

    accuracy                           0.82       814
   macro avg       0.60      0.57      0.58       814
weighted avg       0.81      0.82      0.81       814

Class: PRFN
Accuracy: 259/293

Class: NONE
Accuracy: 380/414

Class: OFFN
Accuracy: 27/82

Class: HATE
Accuracy: 4/25



In [38]:
preds_flat = np.argmax(predictions_test, axis=1).flatten()

In [47]:
def change(x):
  if x != 1:
    return 0
  else:
    return x
preds_mod = [change(x) for x in preds_flat]
true_task1 = df_test['task1'].apply(lambda x: 0 if x=='HOF' else 1).values
print(classification_report(true_task1, preds_mod))

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       423
           1       0.86      0.92      0.89       391

    accuracy                           0.89       814
   macro avg       0.89      0.89      0.89       814
weighted avg       0.89      0.89      0.89       814

